In [ ]:
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
!unzip "/content/drive/MyDrive/IML_CXR.zip" -d "/content"

In [ ]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from random import randint
from scipy.ndimage.interpolation import rotate
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
#from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
#from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
#from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input


In [ ]:
# Open the image form working directory
Outcomes = pd.read_csv("cxr_label_train.csv")
X = np.array([np.array(Image.open('./IML_CXR/' + str(fname) + '.jpg'), dtype='float32') for fname in Outcomes['PATIENT ID']])
#rotate_X = np.array([np.array(Image.open('./IML_CXR/' + str(fname) + '.jpg').rotate(randint(-90, 90)), dtype='float32') for fname in Outcomes['PATIENT ID']])

X = np.repeat(X[:, :, :, np.newaxis], 3, axis=3)
X = preprocess_input(X)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3])
print(X.shape)
Y = Outcomes['hospital_outcome']
#Y = to_categorical(Y)
print(Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
print(Y_train)



(1393, 320, 320, 3)
(1393,)
(975,)


In [ ]:
input_shape = (320, 320, 3)

base_model = VGG16(input_shape = (320, 320, 3), # Shape of our images
                    include_top = False, # Leave out the last fully connected layer
                    weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
# x = Dense(1024, activation='relu')(x)
# x = Dense(512, activation='relu')(x)
# Add a final sigmoid layer for classification
x = Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.summary()
model.compile(optimizer = tf.keras.optimizers.SGD(), loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [ ]:
# training the model for 10 epochs
model.fit(X_train, Y_train, batch_size=64, epochs=10, validation_data=(X_train, Y_train))

Epoch 1/10
16/16 [==============================] - 15s 919ms/step - loss: 127.6327 - accuracy: 0.7293 - val_loss: 48.5381 - val_accuracy: 0.8779
Epoch 2/10
16/16 [==============================] - 14s 921ms/step - loss: 75.4969 - accuracy: 0.8474 - val_loss: 534.3170 - val_accuracy: 0.1467
Epoch 3/10
16/16 [==============================] - 15s 941ms/step - loss: 271.6491 - accuracy: 0.7274 - val_loss: 230.4739 - val_accuracy: 0.8779
Epoch 4/10
16/16 [==============================] - 15s 965ms/step - loss: 88.3020 - accuracy: 0.8974 - val_loss: 93.7359 - val_accuracy: 0.8800
Epoch 5/10
16/16 [==============================] - 15s 988ms/step - loss: 41.6119 - accuracy: 0.8853 - val_loss: 4.5303 - val_accuracy: 0.9651
Epoch 6/10
16/16 [==============================] - 16s 1s/step - loss: 3.9589 - accuracy: 0.9644 - val_loss: 8.4351 - val_accuracy: 0.9436
Epoch 7/10
16/16 [==============================] - 16s 1s/step - loss: 6.9956 - accuracy: 0.9456 - val_loss: 6.2056 - val_accuracy:

In [ ]:
Y_pred = model.predict(X_test)
print(Y_pred)
for i in range(len(Y_pred)):
    if Y_pred[i] > 1e-15:
        Y_pred[i] = 1
    else:
        Y_pred[i] = 0
#Y_test = Y_test.to_list()
TN, FN, TP, FP = 0, 0, 0, 0
for i in range(len(Y_test)):
      if Y_test[i] == 0 and Y_pred[i] == 0:
              TN += 1
      if Y_test[i] == 1 and Y_pred[i] == 0:
              FN += 1
      if Y_test[i] == 0 and Y_pred[i] == 1:
              FP += 1
      if Y_test[i] == 1 and Y_pred[i] == 1:
              TP += 1

print("TN:", TN, ", FN:", FN, ", TP:", TP, ", FP:", FP)
precision, recall = (TP/(FP+TP)), (TP/(FN+TP))
print("precision:", precision, ", recall:", recall)
print('F1:',  2 * ((precision*recall)/(precision+recall)))

[[0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [4.4259288e-15]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.4322484e-22]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+0

In [ ]:
!unzip "/content/drive/MyDrive/IML_CXR_TEST.zip" -d "/content"

In [ ]:
Data_list=[x.split('.')[0] for x in os.listdir('IML_CXR_TEST')]
for i in range(len(Data_list)): 
    Data_list[i] = int(Data_list[i]) 
Data_list.sort()
Test_data = np.array([np.array(Image.open('./IML_CXR_TEST/' + str(fname) + '.jpg'), dtype='float32') for fname in Data_list])
Test_data = np.repeat(Test_data[:, :, :, np.newaxis], 3, axis=3)
Test_data = preprocess_input(Test_data)
Test_data = Test_data.reshape(Test_data.shape[0], Test_data.shape[1], Test_data.shape[2], Test_data.shape[3])

In [ ]:
Y_pred = model.predict(Test_data)
for i in range(len(Y_pred)):
    if Y_pred[i] > 1e-5:
        Y_pred[i] = 1
    else:
        Y_pred[i] = 0
Outcomes = pd.DataFrame(Data_list, columns = ['PATIENT ID']) 
Outcomes['hospital_outcome'] = Y_pred.astype(int)
print(Outcomes)

#Output_data = np.vstack((Data_list, Y_pred.tolist())
Outcomes.to_csv('107062338_' + str(2 * ((precision*recall)/(precision+recall))) + '.csv', index=False)

     PATIENT ID  hospital_outcome
0            51                 0
1            54                 0
2            59                 0
3            60                 0
4            61                 0
..          ...               ...
452        2526                 1
453        2538                 0
454        2554                 0
455        2567                 0
456        2569                 1

[457 rows x 2 columns]
